In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [2]:
import pandas as pd
from datetime import datetime


pos = 0
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges
1,agda,09947741419,dois_riachos


In [3]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [4]:
def people(has_dm):
    
    openEpHealph()
        
    print(f'Extração iniciada as: {datetime.now()}')

    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")
    
    base = nav.find_element(By.XPATH, has_dm).click()

#     children = nav.find_element(By.ID, 'maxAge')
#     nav.execute_script("arguments[0].setAttribute('value', '4.0')",children)
#     time.sleep(1)
    
#     hypertension = nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
#     time.sleep(1)

#     diabetes = nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
#     time.sleep(1)

#     pregnant = nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
#     time.sleep(1)

    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)
    """

    time.sleep(1)

    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()

    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [5]:
def get_table(has_dm):
    
    list_links = []
    people(has_dm)
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            if ElementClickInterceptedException:    
                traceback.print_exc()
            if NoSuchElementException: 
                traceback.print_exc()
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [6]:
def get_individual_info(has_dm, p_has, p_dm):
    
#   name sex b_date cpf cns has dm med_rec race m_name, proffisional, unity
    name, sex, b_date, cpf, cns, has, dm, med_rec, race, m_name, profissional, unity = [], [], [], [], [], [], [], [], [], [], [], []

    
    individual = get_table(has_dm)
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        i_sex = i_sex.split(',')
        sex.append(i_sex[0].upper())
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        i_bdate = i_bdate.split(";")
        b_date.append(i_bdate[0])
        
        linha_dois = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[1]').text
        linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
        
        linha_dois_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
        
        if(linha_dois.upper() == "CPF"):
            cpf.append(linha_dois_text)
            
        elif(linha_tres.upper() == "CPF"):
            cpf.append(linha_tres_text)
            
        else:
            cpf.append("--")
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        i_cns = i_cns.split(':')
        cns.append(i_cns[1])
        
        i_has = p_has
        has.append(i_has)
        
        i_dm = p_dm
        dm.append(i_dm)
        
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
    
        try:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[3]').text
        
        except NoSuchElementException:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        
        i_uni = i_uni.split(':')
        unity.append(i_uni[1])
        
        try:
            linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
            linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
            if(linha_tres.upper() == 'PROFISSIONAL'):
                profissional.append(linha_tres_text)
        except:
            continue
        try:
            linha_quatro = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[1]').text
            linha_quatro_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[2]').text
            if(linha_quatro.upper() == 'PROFISSIONAL'):
                profissional.append(linha_quatro_text)
        except:
            continue
        
        try:
            linha_cinco = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[1]').text
            linha_cinco_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[2]').text
            if(linha_cinco.upper() == 'PROFISSIONAL'):
                profissional.append(linha_cinco_text)
        except:
            continue
        
        try:
            linha_seis = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[1]').text
            linha_seis_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[2]').text
            if(linha_seis.upper() == 'PROFISSIONAL'):
                profissional.append(linha_seis_text)
            
        except:
            continue
            
        time.sleep(1)
        
    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "has":has, "dm":dm, "med_rec":med_rec, "race":race, "m_name":m_name, "profissional":profissional, "unity":unity, "link":individual}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')    
    return df_individual

# indi = get_indivivual_info()

In [7]:
def get_has_dm():
    has_dm_click = ['//*[@id="img_hipertensaoFlag"]', '//*[@id="img_diabetesFlag"]']
    has = get_individual_info(has_dm_click[0], 'sim', 'nao')
    dm = get_individual_info(has_dm_click[1], 'nao', 'sim')
    
    has_dm = pd.concat([has, dm], ignore_index=True)
    return has_dm
    

In [8]:
# unification = get_has_dm()

In [11]:
unic = pd.read_excel('unification.xlsx')
display(unic)

,name,sex,b_date,cpf,cns,has,dm,med_rec,race,m_name,profissional,unity,link
0,ABNER DA SILVA VASCONCELLOS,MASCULINO,28/03/1981,03648041495,700009126573002,sim,nao,--,PARDA,GILDA MARIA DA SILVA VASCONCELOS,ALYSSON TIAGO MARQUES BRANDAO,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
1,ACIDALIA AMORIM VIANA,FEMININO,22/01/1944,58938320430,705007474533958,sim,nao,821,BRANCA,IZABEL MASCENA AMORIM,DANIELLA KELLY LUCAS DE MORAIS RAMOS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
2,ADEILTON MARQUES DE BRITO,MASCULINO,10/11/1965,40301710449,708608165060990,sim,nao,1025,BRANCA,(desconhecido),DANIELLA KELLY LUCAS DE MORAIS RAMOS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
3,ADELMO FREITAS DE BRITO,MASCULINO,03/04/1965,33638500349,700600993771369,sim,nao,203,BRANCA,MARIA DE LOURDES DE FREITAS,IVANEIDE OLIVEIRA DOS SANTOS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
4,ADEMIR RODRIGUES DA SILVA,MASCULINO,05/08/1936,01068954434,700106953790117,sim,nao,01003,BRANCA,FRANCISCA SIMÕES DA SILVA,DANIELLA KELLY LUCAS DE MORAIS RAMOS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,VICENTE JOSÉ FERREIRA ZUZA,MASCULINO,14/12/1974,84498587472,708103569225339,nao,sim,1855,PRETA,MARIA AUXILIADORA FERREIRA,MARIA ZULEIDE BESERRA CANDIDO,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
789,WALDIRENE FERREIRA DE SANTANA,FEMININO,01/09/1967,55794602449,700803983690790,nao,sim,--,PARDA,HELENA FERREIRA DE SOUZA,DANIELLA KELLY LUCAS DE MORAIS RAMOS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
790,WANDERLEY GOMES DE LIMA,MASCULINO,02/10/1973,01922620416,704607760021730,nao,sim,548,BRANCA,MARIA TEREZA DE LIMA,MARIA ZULEIDE BESERRA CANDIDO,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
791,WILMA LUCIA DO NASCIMENTO PAZ,FEMININO,10/03/1975,02574415493,704603140228521,nao,sim,--,PARDA,ALAIDE DO NASCIMENTO,ALYSSON TIAGO MARQUES BRANDAO,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...


In [21]:
double = unic[unic.duplicated(subset=['name', 'cpf'])]


double.index = [x for x in range(0, len(double))]
display(double)

double.to_r

double.to_excel('duplicadas.xlsx', index=False)

,name,sex,b_date,cpf,cns,has,dm,med_rec,race,m_name,profissional,unity,link
0,ADRIANA FERREIRA DA SILVA,FEMININO,29/11/1968,10633481882,706406176042887,nao,sim,--,PARDA,APARECIDA MACHADO DA COSTA,PAULA SUZILANEA NICACIO VERAS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
1,ÁLVARO MARCELO BESERRA DOS RAMOS,MASCULINO,07/05/1961,23887699491,702603212059341,nao,sim,205,BRANCA,ALZIRA BEZERRA DOS RAMOS,IVANEIDE OLIVEIRA DOS SANTOS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
2,AMADEU ESTEVÃO NERES,MASCULINO,13/02/1948,93472560487,704600616135523,nao,sim,--,BRANCA,QUITERIA JOSEFA DA CONCEIÇÃO,MARIA JOSE DE SOUSA ALVES MARQUES,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
3,ANA CRISTINA GOMES,FEMININO,07/09/1973,06005971484,708003881384723,nao,sim,448,PARDA,JUDITE SABINO GOMES,IVANEIDE OLIVEIRA DOS SANTOS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
4,ANGELA NINFA LUCAS DA SILVA,FEMININO,22/12/1961,31875432434,704209218791686,nao,sim,01133,PARDA,MARIA DE LURDES LUCAS DA SILVA,DANIELLA KELLY LUCAS DE MORAIS RAMOS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,VERA LUCIA DA SILVA ALVES,FEMININO,17/04/1971,76447502404,700509789325754,nao,sim,501,PRETA,GENI NICOLAU DA SILVA,MARIA ZULEIDE BESERRA CANDIDO,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
135,VERA LUCIA PEREIRA DE SIQUEIRA,FEMININO,26/04/1965,48523798404,702800154743260,nao,sim,764,BRANCA,DJANIRA MARIA LEITE,MARIA JOSE DE SOUSA ALVES MARQUES,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
136,VICENTE JOSÉ FERREIRA ZUZA,MASCULINO,14/12/1974,84498587472,708103569225339,nao,sim,1855,PRETA,MARIA AUXILIADORA FERREIRA,MARIA ZULEIDE BESERRA CANDIDO,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...
137,WALDIRENE FERREIRA DE SANTANA,FEMININO,01/09/1967,55794602449,700803983690790,nao,sim,--,PARDA,HELENA FERREIRA DE SOUZA,DANIELLA KELLY LUCAS DE MORAIS RAMOS,PSF BORGES,https://afogadosdaingazeira.ephealth.com.br/po...


/tmp/ipykernel_53492/2585884895.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  double['has'] = 'sim'
/tmp/ipykernel_53492/2585884895.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  double['dm'] = 'sim'
